In [1]:
# !pip install kafka-python 

In [1]:
import json
import time
import pandas as pd

from kafka import KafkaProducer, KafkaConsumer


In [2]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:29092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [3]:
df_green = pd.read_csv('https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz', usecols=['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount'])
df_green.head()
# df_green=df_green.head(5)

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26


In [6]:
df_green.shape[0]

476386

In [7]:
topic_name = 'green-trips'

In [8]:
t0 = time.time()

count = 0

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}

    # Перетворення даних в JSON
    json_data = json.dumps(row_dict)

    # Надсилання даних
    producer.send(topic_name, value=json_data)
    count = count + 1

    # Додаткова затримка (необов'язково)
    # time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')
print('total records published', count)

took 64.85 seconds
total records published 476386
